In [2]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.tail()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
19099,2017-08-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,740280,3
19100,2017-09-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,448370,3
19101,2017-10-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,0,2
19102,2017-11-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,200420,2
19103,2017-12-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,411480,2


In [4]:
train['LastWorkingDate'] = train['LastWorkingDate'].fillna(0)
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,0,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,0,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,0,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,0,2,2,0,1


In [5]:
df = pd.DataFrame({'Emp_ID':sorted(train.Emp_ID.unique())})
df.head()

,Emp_ID
0,1
1,2
2,4
3,5
4,6


In [6]:
age_col = train.groupby(['Emp_ID']).Age.agg(['min'])
age_col = age_col.reset_index()
age_col.columns = ['Emp_ID','Age']

def get_age(x):
  return age_col[age_col.Emp_ID==x].Age.iloc[0]

df["Age"] = age_col.Emp_ID.apply(get_age)
df

,Emp_ID,Age
0,1,28
1,2,31
2,4,43
3,5,29
4,6,31
...,...,...
2376,2784,33
2377,2785,34
2378,2786,44
2379,2787,28


In [7]:
gen_col = train.groupby(['Emp_ID','Gender']).Gender.agg('count').to_frame()
gen_col.columns = ['gen_count']
gen_col = gen_col.reset_index()

def get_gen(x):
  return gen_col[gen_col.Emp_ID==x].Gender.iloc[0]

df["Gender"] = gen_col.Emp_ID.apply(get_gen)
df

,Emp_ID,Age,Gender
0,1,28,Male
1,2,31,Male
2,4,43,Male
3,5,29,Male
4,6,31,Female
...,...,...,...
2376,2784,33,Male
2377,2785,34,Female
2378,2786,44,Male
2379,2787,28,Female


In [8]:
city_col = train.groupby(['Emp_ID','City']).City.agg('count').to_frame()
city_col.columns = ['city_count']
city_col = city_col.reset_index()

def get_city(x):
  return city_col[city_col.Emp_ID==x].City.iloc[0]

df["City"] = city_col.Emp_ID.apply(get_city)
df

,Emp_ID,Age,Gender,City
0,1,28,Male,C23
1,2,31,Male,C7
2,4,43,Male,C13
3,5,29,Male,C9
4,6,31,Female,C11
...,...,...,...,...
2376,2784,33,Male,C24
2377,2785,34,Female,C9
2378,2786,44,Male,C19
2379,2787,28,Female,C20


In [9]:
edu_col = train.groupby(['Emp_ID','Education_Level']).Education_Level.agg('count').to_frame()
edu_col.columns = ['edu_count']
edu_col = edu_col.reset_index()

def get_edu(x):
  return edu_col[edu_col.Emp_ID==x].Education_Level.iloc[0]

df["Education_Level"] = edu_col.Emp_ID.apply(get_edu)
df

,Emp_ID,Age,Gender,City,Education_Level
0,1,28,Male,C23,Master
1,2,31,Male,C7,Master
2,4,43,Male,C13,Master
3,5,29,Male,C9,College
4,6,31,Female,C11,Bachelor
...,...,...,...,...,...
2376,2784,33,Male,C24,College
2377,2785,34,Female,C9,College
2378,2786,44,Male,C19,College
2379,2787,28,Female,C20,Master


In [10]:
sal_col = train.groupby(['Emp_ID']).Salary.agg(['max'])
sal_col = sal_col.reset_index()
sal_col.columns = ['Emp_ID','Salary']

def get_sal(x):
  return sal_col[sal_col.Emp_ID==x].Salary.iloc[0]

df["Salary"] = sal_col.Emp_ID.apply(get_sal)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary
0,1,28,Male,C23,Master,57387
1,2,31,Male,C7,Master,67016
2,4,43,Male,C13,Master,65603
3,5,29,Male,C9,College,46368
4,6,31,Female,C11,Bachelor,78728
...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815
2377,2785,34,Female,C9,College,12105
2378,2786,44,Male,C19,College,35370
2379,2787,28,Female,C20,Master,69498


In [11]:
all_occur = train.groupby(['Emp_ID']).size().to_frame()
all_occur = all_occur.reset_index()
all_occur.columns = ['Emp_ID','months_worked']
def months_worked_baby(x):
  return all_occur[all_occur.Emp_ID==x].months_worked.iloc[0]

df["months_worked"] = all_occur.Emp_ID.apply(months_worked_baby)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary,months_worked
0,1,28,Male,C23,Master,57387,3
1,2,31,Male,C7,Master,67016,2
2,4,43,Male,C13,Master,65603,5
3,5,29,Male,C9,College,46368,3
4,6,31,Female,C11,Bachelor,78728,5
...,...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815,24
2377,2785,34,Female,C9,College,12105,3
2378,2786,44,Male,C19,College,35370,9
2379,2787,28,Female,C20,Master,69498,6


In [12]:
join_des = train.groupby(['Emp_ID'])['Joining Designation'].agg(['max'])
join_des = join_des.reset_index()
join_des.columns = ['Emp_ID','Joining_Designation']
join_des

def get_jdes(x):
  return join_des[join_des.Emp_ID==x].Joining_Designation.iloc[0]

df["Joining_Designation"] = join_des.Emp_ID.apply(get_jdes)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary,months_worked,Joining_Designation
0,1,28,Male,C23,Master,57387,3,1
1,2,31,Male,C7,Master,67016,2,2
2,4,43,Male,C13,Master,65603,5,2
3,5,29,Male,C9,College,46368,3,1
4,6,31,Female,C11,Bachelor,78728,5,3
...,...,...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815,24,2
2377,2785,34,Female,C9,College,12105,3,1
2378,2786,44,Male,C19,College,35370,9,2
2379,2787,28,Female,C20,Master,69498,6,1


In [13]:
cur_des = train.groupby(['Emp_ID'])['Designation'].agg(['max'])
cur_des = cur_des.reset_index()
cur_des.columns = ['Emp_ID','Designation']
cur_des
def get_cdes(x):
  return cur_des[cur_des.Emp_ID==x].Designation.iloc[0]

df["Designation"] = cur_des.Emp_ID.apply(get_cdes)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary,months_worked,Joining_Designation,Designation
0,1,28,Male,C23,Master,57387,3,1,1
1,2,31,Male,C7,Master,67016,2,2,2
2,4,43,Male,C13,Master,65603,5,2,2
3,5,29,Male,C9,College,46368,3,1,1
4,6,31,Female,C11,Bachelor,78728,5,3,3
...,...,...,...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815,24,2,3
2377,2785,34,Female,C9,College,12105,3,1,1
2378,2786,44,Male,C19,College,35370,9,2,2
2379,2787,28,Female,C20,Master,69498,6,1,1


In [14]:
qua_rat = train.groupby(['Emp_ID'])['Quarterly Rating'].agg(['min'])
qua_rat = qua_rat.reset_index()
qua_rat.columns = ['Emp_ID','Minimum_Quarterly_Rating']
qua_rat

def get_rat(x):
  return qua_rat[qua_rat.Emp_ID==x].Minimum_Quarterly_Rating.iloc[0]

df["Minimum_Quarterly_Rating"] = qua_rat.Emp_ID.apply(get_rat)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary,months_worked,Joining_Designation,Designation,Minimum_Quarterly_Rating
0,1,28,Male,C23,Master,57387,3,1,1,2
1,2,31,Male,C7,Master,67016,2,2,2,1
2,4,43,Male,C13,Master,65603,5,2,2,1
3,5,29,Male,C9,College,46368,3,1,1,1
4,6,31,Female,C11,Bachelor,78728,5,3,3,1
...,...,...,...,...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815,24,2,3,1
2377,2785,34,Female,C9,College,12105,3,1,1,1
2378,2786,44,Male,C19,College,35370,9,2,2,1
2379,2787,28,Female,C20,Master,69498,6,1,1,1


In [15]:
qua_rat = train.groupby(['Emp_ID'])['Quarterly Rating'].agg(['max'])
qua_rat = qua_rat.reset_index()
qua_rat.columns = ['Emp_ID','Maximum_Quarterly_Rating']
qua_rat

def get_rat(x):
  return qua_rat[qua_rat.Emp_ID==x].Maximum_Quarterly_Rating.iloc[0]

df["Maximum_Quarterly_Rating"] = qua_rat.Emp_ID.apply(get_rat)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary,months_worked,Joining_Designation,Designation,Minimum_Quarterly_Rating,Maximum_Quarterly_Rating
0,1,28,Male,C23,Master,57387,3,1,1,2,2
1,2,31,Male,C7,Master,67016,2,2,2,1,1
2,4,43,Male,C13,Master,65603,5,2,2,1,1
3,5,29,Male,C9,College,46368,3,1,1,1,1
4,6,31,Female,C11,Bachelor,78728,5,3,3,1,2
...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815,24,2,3,1,4
2377,2785,34,Female,C9,College,12105,3,1,1,1,1
2378,2786,44,Male,C19,College,35370,9,2,2,1,2
2379,2787,28,Female,C20,Master,69498,6,1,1,1,2


In [16]:
doj = train.groupby(['Emp_ID','Dateofjoining']).Dateofjoining.agg('count').to_frame()
doj.columns = ['doj_count']
doj = doj.reset_index()

def get_doj(x):
  return doj[doj.Emp_ID==x].Dateofjoining.iloc[0]

df["Dateofjoining"] = doj.Emp_ID.apply(get_doj)
df

,Emp_ID,Age,Gender,City,Education_Level,Salary,months_worked,Joining_Designation,Designation,Minimum_Quarterly_Rating,Maximum_Quarterly_Rating,Dateofjoining
0,1,28,Male,C23,Master,57387,3,1,1,2,2,2015-12-24
1,2,31,Male,C7,Master,67016,2,2,2,1,1,2017-11-06
2,4,43,Male,C13,Master,65603,5,2,2,1,1,2016-12-07
3,5,29,Male,C9,College,46368,3,1,1,1,1,2016-01-09
4,6,31,Female,C11,Bachelor,78728,5,3,3,1,2,2017-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,33,Male,C24,College,82815,24,2,3,1,4,2012-10-15
2377,2785,34,Female,C9,College,12105,3,1,1,1,1,2017-08-28
2378,2786,44,Male,C19,College,35370,9,2,2,1,2,2015-07-31
2379,2787,28,Female,C20,Master,69498,6,1,1,1,2,2015-07-21


In [17]:
X_train,X_val, y_train,y_val = train_test_split(X,y,random_state=42,test_size=0.2)

NameError: name 'X' is not defined

In [ ]:
model = CatBoostRegressor(loss_function='RMSE')
model.fit(X_train,y_train,cat_features=['Gender','City','Education_Level','Dateofjoining'])

In [ ]:
y_pred_val = model.predict(X_val)

In [ ]:
r2_score(y_pred_val,y_val)

In [ ]:
# df = df.drop('Dateofjoining',axis=1)


In [ ]:
df
X = df.drop('months_worked',axis=1)
y = df['months_worked']

In [ ]:
model.fit(X,y,cat_features=['Gender','City','Education_Level'])

In [ ]:
merged_right = pd.merge(left=df, right=test, how='right', left_on='Emp_ID', right_on='Emp_ID')
merged_right

In [ ]:
y_pred = model.predict(merged_right)

In [ ]:
mysub = pd.read_csv('sample_submission_znWiLZ4.csv')

In [ ]:
mysub.Target=y_pred

In [ ]:
mysub.to_csv('mysub.csv',index=False)